In [ ]:
import numpy as np
import torch 
import torchvision 
import torchaudio
import torch.nn as nn
import math
import torch.nn.functional as F
from Jaguas_DataLoader import SoundscapeData
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import random_split
!pip install positional-encodings[pytorch]
from positional_encodings.torch_encodings import PositionalEncoding1D, PositionalEncoding2D, PositionalEncoding3D, Summer, PositionalEncodingPermute2D
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
p_enc_2d = PositionalEncodingPermute2D(5)
y = torch.zeros((1,6,2,8))
print(p_enc_2d(y).shape) # (1, 6, 2, 8)
p_enc_2d(y)

In [ ]:
class PositionalEncoding2d(nn.Module):

    def __init__(self, d_model: int, height: int = 515, width: int =515, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(d_model, height, width)
        # Each dimension use half of d_model
        d_model = int(d_model / 2)
        div_term = torch.exp(torch.arange(0., d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pos_w = torch.arange(0., width).unsqueeze(1)
        pos_h = torch.arange(0., height).unsqueeze(1)
        pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
        pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
        pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
        pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
        self.register_buffer('pe', pe)
        
    def forward(self, x, index: int, dropout: bool=False):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x.to("cuda")
        self.pe = self.pe.to("cuda")
        x = x + self.pe[index]
        if dropout:
            x = self.dropout(x)
        else:
            x = x
        return x

In [ ]:
class posautoencoding_m1(nn.Module):

    """
    Convolutional autoencoder made to reconstruct the audios spectrograms generated by the EcoDataTesis dataloader.
    """

    def __init__(self, num_hiddens: int=64):
        """
        Constructor of the convolutional autoencoder model.
        """
        super().__init__()
        # TODO: To design the final architechture considering the spectrograms sizes.
        # TODO: To correct the current sizes of the decoder.

        self.encoder = nn.Sequential(
            nn.Conv2d(1, num_hiddens // 8, kernel_size=8, stride=3, padding=0),  # N, 256, 127, 8004
            nn.ReLU(),
            nn.Conv2d(num_hiddens // 8, num_hiddens // 4, kernel_size=8, stride=3, padding=0),  # N, 512, 125,969
            nn.ReLU(),
            nn.Conv2d(num_hiddens // 4, num_hiddens // 2, kernel_size=4, stride=3, padding=0),  # N, 512, 125,969
            nn.ReLU(),
            nn.Conv2d(num_hiddens // 2, num_hiddens, kernel_size=2, stride=2, padding=0),  # N, 512, 125,969
            nn.ReLU()
             )
        self.decoder = nn.Sequential(  # This is like go in opposite direction respect the encoder
            nn.ConvTranspose2d(num_hiddens, num_hiddens // 2, kernel_size=2, stride=2, padding=0, output_padding=0),  # N, 32, 126,8000
            nn.ReLU(),
            nn.ConvTranspose2d(num_hiddens // 2, num_hiddens // 4, kernel_size=4, stride=3, padding=0, output_padding=0),  # N, 32, 127,64248
            nn.ReLU(),
            nn.ConvTranspose2d(num_hiddens // 4, num_hiddens // 8, kernel_size=8, stride=3, padding=0, output_padding=0),  # N, 32, 127,64248
            nn.ReLU(),
            nn.ConvTranspose2d(num_hiddens // 8, 1, kernel_size=8, stride=3, padding=0, output_padding=0),  # N, 32, 127,64248
            nn.Sigmoid()
             )
            
#         self.posencoding = PositionalEncodingPermute2D(5)
            
            
    def forward(self, x, y):
        
        """
        Method to compute an image output based on the performed model.

        :param x: Input spectrogram images as tensors.
        :type x: torch.tensor
        :return: Reconstructed images
        """
        
        #print(f"x_shape:{x.shape}")
        pos_encoder = PositionalEncoding2d(64, dropout = 0.1, max_len = 20).to("cuda")
        posencoding_2d = pos_encoder(x, y) 
        encoded = self.encoder(x)
        penc_no_sum = posencoding_2d
        print("encoder_shape: ", encoded.shape)
        decoded = self.decoder(encoded)
        print("decoder_shape: ",decoded.shape)
        return decoded, penc_no_sum

In [ ]:
root_path = 'media/mirp_ai/DATA1/Jaguas_2018'
dataset = SoundscapeData(root_path, audio_length=12, ext="wav", win_length=1028)
dataset_train, dataset_test = random_split(dataset,
                                           [round(len(dataset)*0.7), len(dataset) - round(len(dataset)*0.7)], 
                                           generator=torch.Generator().manual_seed(1024))

config = {
    "project" : "positionalAE-Jaguas",
    "audio_length": dataset.audio_length,
    "batch_size" : 14,
    "num_epochs": 3,
    "num_hiddens" : 64,
    "gamma_lr" : 0.1,
    "learning_rate" : 1e-3,
    "dataset" : "Audios Jaguas",
    "architecture": "AE",
    "win_length" : dataset.win_length
}

training_loader = DataLoader(dataset_train, batch_size=config["batch_size"])
test_loader = DataLoader(dataset_test, batch_size=config["batch_size"])

model = posautoencoding_m1(num_hiddens=config["num_hiddens"]).to("cuda")

optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], amsgrad=False)
scheduler = lr_scheduler.StepLR(optimizer, step_size = 6, gamma = config["gamma_lr"] )

config["optimizer"] = optimizer
config["scheduler"] = scheduler
config["num_training_updates"] = len(training_loader)

In [ ]:
iterador = iter(training_loader)
data, b, c, d = next(iterador)
data = torch.reshape(data, (data.shape[0] * data.shape[1] * data.shape[2], data.shape[3], data.shape[4]))
data = torch.unsqueeze(data, 1)
data.shape

In [ ]:
from six.moves import xrange
class TrainModel:

    def __init__(self, model):
        self._model = model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self._model.to(self.device)
        print(self.device)


    def forward(self, training_loader, test_loader, config):
        iterator = iter(test_loader)
        run_name = "posae"
        optimizer = config["optimizer"]
        scheduler = config["scheduler"]
        
        logs = []
        # best_loss = 10000

        for epoch in range(config["num_epochs"]):
            iterator_train = iter(training_loader)
            for i in xrange(config["num_training_updates"]):
                self._model.train()
                try:
                    data, _, label, _ = next(iterator_train)
                except Exception as e:
                    print("error")
                    print(e)
                    logs.append(e)
                    continue

                data = torch.reshape(data, (data.shape[0] * data.shape[1] * data.shape[2], data.shape[3], data.shape[4]))
                data = torch.unsqueeze(data, 1)
                data = data.to("cuda")

                optimizer.zero_grad()
                data_recon, _ = self._model(data, label["recorder"].reshape(14*5))

                loss = F.mse_loss(data_recon, data)
                loss.backward()

                optimizer.step()
                print(
                    f'epoch: {epoch + 1} of {config["num_epochs"]} \t iteration: {(i + 1)} of {config["num_training_updates"]} \t loss: {np.round(loss.item(), 4)}')
                dict = {"loss": loss.item()}

                if loss < 0.04:
                    # wandb.alert(
                    #     title='High accuracy',
                    #     text=f'Recon error {loss} is lower than 0.04',
                    #     level=AlertLevel.WARN,
                    #     wait_duration=timedelta(minutes=1)
                    # )
                    time = datetime.datetime.now()
                    torch.save(self._model.state_dict(), f'{run_name}_day_{time.day}_hour_{time.hour}_low_error.pkl')
                else:
                    pass

            scheduler.step()
            torch.cuda.empty_cache()
            time = datetime.datetime.now()
            torch.save(self._model.state_dict(), f'{run_name}_day_{time.day}_hour_{time.hour}_epoch_{epoch + 1}.pkl')
            clear_output()
            print(optimizer.state_dict()["param_groups"][0]["lr"])

#         wandb.finish()
        return self._model, logs, run_name

In [ ]:
Training = TrainModel(model=model.to("cuda"))
model, logs, run_name = Training.forward(training_loader, test_loader, config)
time = datetime.datetime.now()
torch.save(model.state_dict(),f'temporal/models/model_{run_name}_day_{time.day}_hour_{time.hour}_final.pth')
torch.save(config,f'temporal/configs/config_{run_name}_day_{time.day}_hour_{time.hour}.pth')
torch.save(dataset_test, f"temporal/datasets/dataset_test_ae_jaguas_{time.day}_70%.pth")
torch.save(dataset_train, f"temporal/datasets/dataset_train_ae_jaguas_{time.day}_70%.pth")

In [ ]:
train.forward(training_loader, test_loader, config)

In [ ]:
c["recorder"].reshape(14*5)